In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

## Hyperparameter Optimization Experiment Definition

In [ ]:
MOCKUP = False #if to use mockup components in experiment (for testing general logic without having to do a lot of computations)

In [ ]:
base_experiment_path = "data\\experiments"

In [ ]:
from automl.rl.environment.gymnasium_env import GymnasiumEnvironmentWrapper

# Choose environment to test

environment_to_test = (GymnasiumEnvironmentWrapper, {})
#environment_to_test = None # this would mean to use the default


In [ ]:
# To help distinguish experiments using the environment

#base_experiment_path = os.path.join(base_experiment_path, "CartPole")
base_experiment_path = os.path.join(base_experiment_path, "MountainCar")


In [ ]:
# Which base experiment to follow

import scripts.basic_dqn_hp_experiment as experiment
#import scripts.basic_ppo_hp_experiment as experiment

In [ ]:
# to help distinguish experiments using specifications of algorithms
base_experiment_path = os.path.join(base_experiment_path, "DQN")

In [ ]:
hp_optimization_input = {
    "n_trials" : 40,
    "steps" : 2,
    "base_directory" :  base_experiment_path,
    }

In [ ]:
# If you want to generate the hyperparameters to change


hyperparameters_to_change = None

#hyperparameters_to_change : list = experiment.get_hyperparameters_to_change()

if hyperparameters_to_change != None:
    hp_optimization_input["hyperparameters_to_change"] = hyperparameters_to_change

In [ ]:
# If you want to pass the component to optimize by giving directly its configuration dict


configuration_dict = None

#configuration_dict = experiment.get_configuration_dict(num_episodes=500, env=environment_to_test, mockup=MOCKUP)


# from automl.base_configurations.environment.montain_car.dqn_sb3 import config_dict as dqn_sb3_config_dict
#configuration_dict = dqn_sb3_config_dict()

In [ ]:
# If you want to pass the component to optimize by giving directly its path

base_component_configuration_path = None

base_component_configuration_path = "data\\experiments\\MountainCar\\DQN\\to_optimize_configuration.json"


In [ ]:
from automl.rl.evaluators.rl_std_avg_evaluator import LastValuesAvgStdEvaluator
from automl.rl.evaluators.rl_evaluator_player import EvaluatorWithPlayer


#evaluator = None
#evaluator = (LastValuesAvgStdEvaluator, {"value_to_use" : "episode_steps"})
evaluator = (EvaluatorWithPlayer, {"base_evaluator" : (LastValuesAvgStdEvaluator, {"value_to_use" : "episode_steps"})})

In [ ]:
# Generate Pruner

import optuna


pruner = None
pruner = optuna.pruners.PercentilePruner(percentile=25.0)

if pruner != None:
    hp_optimization_input["pruner"] = pruner

In [ ]:
if configuration_dict != None:
    hp_optimization_input["configuration_dict"] = configuration_dict
    
    
if base_component_configuration_path != None: 
    hp_optimization_input["base_component_configuration_path"] = base_component_configuration_path

In [ ]:
# If you want to load the basic configuration of the hyperparameter optimization pipeline

hp_optimization_pipeline_path = None

hp_optimization_pipeline_path = "data\\experiments\\MountainCar\\DQN\\configuration.json"

In [ ]:
from automl.meta_rl.hp_optimization_pipeline import HyperparameterOptimizationPipeline

if hp_optimization_pipeline_path != None: # load the configuration in the path
    
    from automl.utils.json_component_utils import gen_component_from_path
    hp_optimization_pipeline = gen_component_from_path(hp_optimization_pipeline_path)

else: # if not, generate empty
    hp_optimization_pipeline = HyperparameterOptimizationPipeline()
    
    
hp_optimization_pipeline.pass_input(hp_optimization_input)


In [ ]:
hp_optimization_pipeline.setup_files() # partially initialize the hp optimization pipeline with the current input, it can be manually changed before it is used

In [ ]:
experiment_directory = hp_optimization_pipeline.get_artifact_directory()

After this, we can make manual changes to the generated json files:

 - to_optimize_configuration.json, which is the base in which we want to optimize the hyperparameters of
 - configuration.json, the definition of the actual hyperparameter optimization experiment 


## Running the Hyperparameter Configuration Experiment

In [ ]:
from automl.basic_components.state_management import load_component_from_folder

# load the defined experiment with its changes
hp_optimization_pipeline : HyperparameterOptimizationPipeline = load_component_from_folder(experiment_directory)

In [ ]:
#import torch
#torch.autograd.set_detect_anomaly(True) # use this in case of certain types of errors

hp_optimization_pipeline.run()

In [ ]:
from automl.basic_components.state_management import save_state

save_state(hp_optimization_pipeline)